#  Highly Performant TensorFlow Batch Inference and Training  

In this notebook, we'll show how to use SageMaker batch transform to get inferences on a large datasets. To do this, we'll use a TensorFlow Serving model to do batch inference on a large dataset of images. We'll show how to use the new pre-processing and post-processing feature of the TensorFlow Serving container on Amazon SageMaker so that your TensorFlow model can make inferences directly on data in S3, and save post-processed inferences to S3.

The dataset we'll be using is the [“Challenge 2018/2019"](https://github.com/cvdfoundation/open-images-dataset#download-the-open-images-challenge-28182019-test-set)” subset of the [Open Images V5 Dataset](https://storage.googleapis.com/openimages/web/index.html). This subset consists of 100,00 images in .jpg format, for a total of 10GB. For demonstration, the [model](https://github.com/tensorflow/models/tree/master/official/resnet#pre-trained-model) we'll be using is an image classification model based on the ResNet-50 architecture that has been trained on the ImageNet dataset, and which has been exported as a TensorFlow SavedModel.

We will use this model to predict the class that each model belongs to. We'll write a pre- and post-processing script and package the script with our TensorFlow SavedModel, and demonstrate how to get inferences on large datasets with SageMaker batch transform quickly, efficiently, and at scale, on GPU-accelerated instances.

## Setup 

We'll begin with some necessary imports, and get an Amazon SageMaker session to help perform certain tasks, as well as an IAM role with the necessary permissions.

In [1]:
import numpy as np
import os
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()
role = get_execution_role()

region = sagemaker_session.boto_region_name
bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/DEMO-tf-batch-inference-jpeg-images-python-sdk'
print('Region: {}'.format(region))
print('S3 URI: s3://{}/{}'.format(bucket, prefix))
print('Role:   {}'.format(role))

Region: us-west-2
S3 URI: s3://sagemaker-us-west-2-038453126632/sagemaker/DEMO-tf-batch-inference-jpeg-images-python-sdk
Role:   arn:aws:iam::038453126632:role/service-role/AmazonSageMaker-ExecutionRole-20180718T141171


## Inspecting the SavedModel

In order to make inferences, we'll have to preprocess our image data in S3 to match the serving signature of our TensorFlow SavedModel (https://www.tensorflow.org/guide/saved_model), which we can inspect using the saved_model_cli (https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/tools/saved_model_cli.py).  This is the serving signature of the ResNet-50 v2 (NCHW, JPEG) (https://github.com/tensorflow/models/tree/master/official/resnet#pre-trained-model) model:

In [2]:
!aws s3 cp s3://sagemaker-sample-data-{region}/batch-transform/open-images/model/resnet_v2_fp32_savedmodel_NCHW_jpg.tar.gz .
!tar -zxf resnet_v2_fp32_savedmodel_NCHW_jpg.tar.gz
!saved_model_cli show --dir resnet_v2_fp32_savedmodel_NCHW_jpg/1538687370/ --all

download: s3://sagemaker-sample-data-us-west-2/batch-transform/open-images/model/resnet_v2_fp32_savedmodel_NCHW_jpg.tar.gz to ./resnet_v2_fp32_savedmodel_NCHW_jpg.tar.gz

MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['predict']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['image_bytes'] tensor_info:
        dtype: DT_STRING
        shape: (-1)
        name: input_tensor:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['classes'] tensor_info:
        dtype: DT_INT64
        shape: (-1)
        name: ArgMax:0
    outputs['probabilities'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 1001)
        name: softmax_tensor:0
  Method name is: tensorflow/serving/predict

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['image_bytes'] tensor_info:
        dtype: DT_STRING
        shape: (-1)
        name: inpu

The SageMaker TensorFlow Serving Container uses the model’s SignatureDef named serving_default , which is declared when the TensorFlow SavedModel is exported. This SignatureDef says that the model accepts a string of arbitrary length as input, and responds with classes and their probabilities. With our image classification model, the input string will be a base-64 encoded string representing a JPEG image, which our SavedModel will decode.

## Writing a pre- and post-processing script

We will package up our SavedModel with a Python script named `inference.py`, which will pre-process input data going from S3 to our TensorFlow Serving model, and post-process output data before it is saved back to S3:

In [3]:
!pygmentize code/inference.py

# Copyright 2018 Amazon.com, Inc. or its affiliates. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License"). You
# may not use this file except in compliance with the License. A copy of
# the License is located at
#
#     http://aws.amazon.com/apache2.0/
#
# or in the "license" file accompanying this file. This file is
# distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF
# ANY KIND, either express or implied. See the License for the specific
# language governing permissions and limitations under the License.

import base64
import io
import json
import requests

def input_handler(data, context):
    """ Pre-process request input before it is sent to TensorFlow Serving REST API

    Args:
        data (obj): the request data stream
        context (Context): an object containing request and configuration details

    Returns:
        (dict): a JSON-serializable dict that contains request body and headers
    """

    if context.request_c

The input_handler intercepts inference requests, base-64 encodes the request body, and formats the request body to conform to TensorFlow Serving’s REST API (https://www.tensorflow.org/tfx/serving/api_rest). The return value of the input_handler function is used as the request body in the TensorFlow Serving request.

Binary data must use key "b64", according to the TFS REST API (https://www.tensorflow.org/tfx/serving/api_rest#encoding_binary_values), and since our serving signature’s input tensor has the suffix "\_bytes", the encoded image data under key "b64" will be passed to the "image\_bytes" tensor. Some serving signatures may accept a tensor of floats or integers instead of a base-64 encoded string, but for binary data (including image data), it is recommended that your SavedModel accept a base-64 encoded string for binary data, since JSON representations of binary data can be large.

Each incoming request originally contains a serialized JPEG image in its request body, and after passing through the input_handler, the request body contains the following, which our TensorFlow Serving accepts for inference:

`{"instances": [{"b64":"[base-64 encoded JPEG image]"}]}`

The first field in the return value of `output_handler` is what SageMaker Batch Transform will save to S3 as this example’s prediction. In this case, our `output_handler` passes the content on to S3 unmodified.

Pre- and post-processing functions let you perform inference with TensorFlow Serving on any data format, not just images. To learn more about the `input_handler` and `output_handler`, consult the SageMaker TensorFlow Serving Container README (https://github.com/aws/sagemaker-tensorflow-serving-container/blob/master/README.md).

## Packaging a Model

After writing a pre- and post-processing script, you’ll need to package your TensorFlow SavedModel along with your script into a `model.tar.gz` file, which we’ll upload to S3 for the SageMaker TensorFlow Serving Container to use. Let's package the SavedModel with the `inference.py` script and examine the expected format of the `model.tar.gz` file:

In [4]:
!tar -cvzf model.tar.gz code --directory=resnet_v2_fp32_savedmodel_NCHW_jpg 1538687370

code/
code/.ipynb_checkpoints/
code/inference.py
1538687370/
1538687370/saved_model.pb
1538687370/variables/
1538687370/variables/variables.data-00000-of-00001
tar: 1538687370/variables/variables.data-00000-of-00001: file changed as we read it
1538687370/variables/variables.index
tar: 1538687370/variables: file changed as we read it
tar: 1538687370: file changed as we read it


`1538687370` refers to the model version number of the SavedModel, and this directory contains our SavedModel artifacts. The code directory contains our pre- and post-processing script, which must be named `inference.py`. I can also include an optional `requirements.txt` file, which is used to install dependencies with `pip` from the Python Package Index before the Transform Job starts, but we don’t need any additional dependencies in this case, so we don't include a requirements file.

We will use this `model.tar.gz` when we create a SageMaker Model, which we will use to run Transform Jobs. To learn more about packaging a model, you can consult the SageMaker TensorFlow Serving Container [README](https://github.com/aws/sagemaker-tensorflow-serving-container/blob/master/README.md).

## Run a Batch Transform job

Next, we'll run a Batch Transform job using our data processing script and GPU-based Amazon SageMaker Model. More specifically, we'll perform inference on a cluster of two instances, though we can choose more or fewer. The objects in the S3 path will be distributed between the instances.

Before we create a Transform Job, let's inspect some of our input data. Here's an example, the first image in our dataset:



<img src="sample_image/00000b4dcff7f799.jpg">

The data in the input path consists of 100,000 JPEG images of varying sizes and shapes. Here is a subset:

In [5]:
!aws s3 ls s3://sagemaker-sample-data-{region}/batch-transform/open-images/jpg/000 --human-readable

2019-07-09 22:19:18  126.2 KiB 00000b4dcff7f799.jpg
2019-07-09 22:19:18  115.8 KiB 00001a21632de752.jpg
2019-07-09 22:19:18  151.0 KiB 0000d67245642c5f.jpg
2019-07-09 22:19:18  159.9 KiB 0001244aa8ed3099.jpg
2019-07-09 22:19:18  115.0 KiB 000172d1dd1adce0.jpg
2019-07-09 22:19:18   65.4 KiB 0001c8fbfb30d3a6.jpg
2019-07-09 22:19:18   70.4 KiB 0001dd930912683d.jpg
2019-07-09 22:19:18   73.0 KiB 0002c96937fae3b3.jpg
2019-07-09 22:19:18  109.2 KiB 0002f94fe2d2eb9f.jpg
2019-07-09 22:19:18  119.2 KiB 000305ba209270dc.jpg
2019-07-09 22:19:18  119.5 KiB 000313fed9979d24.jpg
2019-07-09 22:19:18   77.2 KiB 0003a523fa9b2a3f.jpg
2019-07-09 22:19:18   84.9 KiB 0003d1c3be9ed3d6.jpg
2019-07-09 22:19:18   82.9 KiB 000455be7b222c04.jpg
2019-07-09 22:19:18  104.8 KiB 0004fdbc5b94c7c2.jpg
2019-07-09 22:19:18  144.0 KiB 0005339c44e6071b.jpg
2019-07-09 22:19:19   75.2 KiB 0005aea8c9144c77.jpg
2019-07-09 22:19:19   71.0 KiB 0005facd8dbaf39a.jpg
2019-07-09 22:19:19   47.5 KiB 0005fcc2fd4eaa4a.jpg
2019-07-09 2

### Creating a Model and Running a Transform Job

The code below creates a SageMaker Model entity that will be used for Batch inference, and runs a Transform Job using that Model. The Model contains a reference to the TFS container, and the `model.tar.gz` containing our TensorFlow SavedModel and the pre- and post-processing `inference.py` script.

After we create a SageMaker Model, we use it to run batch predictions using Batch Transform. We specify the input S3 data, content type of the input data, the output S3 data, and instance type and count.

### Performance

For improved performance, we specify two additional parameters `max_concurrent_transforms` and `max_payload`, which control the maximum number of parallel requests that can be sent to each instance in a transform job at a time, and the maximum size of each request body.

When performing inference on entire S3 objects that cannot be split by newline characters, such as images, it is recommended that you set `max_payload` to be slightly larger than the largest S3 object in your dataset, and that you experiment with the `max_concurrent_transforms` parameter in powers of two to find a value that maximizes throughput for your model. For example, we’ve set `max_concurrent_transforms` to 64 after experimenting with powers of two, and we set `max_payload` to 1, since the largest object in our S3 input is less than one megabyte.

In [6]:
%%bash -s "$bucket" "$prefix" "$role" "$region"
# For convenience, we pass in bucket, prefix, role, and region set in first Python set-up cell

BUCKET=$1
PREFIX=$2
ROLE_ARN=$3
REGION=$4

timestamp() {
  date +%Y-%m-%d-%H-%M-%S
}

# Creating the SageMaker Model: 
MODEL_NAME="image-classification-tfs-$(timestamp)"
MODEL_DATA_URL="s3://$BUCKET/$PREFIX/model/model.tar.gz"

aws s3 cp model.tar.gz $MODEL_DATA_URL

# This image is maintained at https://github.com/aws/sagemaker-tensorflow-serving-container
TFS_VERSION="1.13"
PROCESSOR_TYPE="gpu"
IMAGE="520713654638.dkr.ecr.$REGION.amazonaws.com/sagemaker-tensorflow-serving:$TFS_VERSION-$PROCESSOR_TYPE"

aws sagemaker create-model \
    --model-name $MODEL_NAME \
    --primary-container Image=$IMAGE,ModelDataUrl=$MODEL_DATA_URL \
    --execution-role-arn $ROLE_ARN

# Creating the Transform Job:
TRANSFORM_JOB_NAME="tfs-image-classification-job-$(timestamp)"

# Specify where to get input data and where to get output data:
TRANSFORM_S3_INPUT="s3://sagemaker-sample-data-$REGION/batch-transform/open-images/jpg"
TRANSFORM_S3_OUTPUT="s3://$BUCKET/$PREFIX/output"

# Our inference script validates the Content-Type, so we set it to "application/x-image".
# SageMaker Model containers can use the content type field to transform multiple different data formats.
# The Data Source tells Batch to get all objects under the S3 prefix.
TRANSFORM_INPUT_DATA_SOURCE={S3DataSource={S3DataType="S3Prefix",S3Uri=$TRANSFORM_S3_INPUT}}
CONTENT_TYPE="application/x-image"

# Specify resources used to transform the job
INSTANCE_TYPE="ml.p2.xlarge"
INSTANCE_COUNT=2

# Performance parameters. MaxPayloadInMB specifies how large each request body can be.
# Our images happen to be less than 1MB, so we set MaxPayloadInMB to 1MB.
# MaxConcurrentTransforms configures the number of concurrent requests made to the container at once.
# The ideal number depends on the payload size, instance type, and model, so some experimentation
# may be beneficial.
MAX_PAYLOAD_IN_MB=1
MAX_CONCURRENT_TRANSFORMS=64

aws sagemaker create-transform-job \
    --model-name $MODEL_NAME \
    --transform-input DataSource=$TRANSFORM_INPUT_DATA_SOURCE,ContentType=$CONTENT_TYPE \
    --transform-output S3OutputPath=$TRANSFORM_S3_OUTPUT \
    --transform-resources InstanceType=$INSTANCE_TYPE,InstanceCount=$INSTANCE_COUNT \
    --max-payload-in-mb $MAX_PAYLOAD_IN_MB \
    --max-concurrent-transforms $MAX_CONCURRENT_TRANSFORMS \
    --transform-job-name $TRANSFORM_JOB_NAME \


echo "Model name: $MODEL_NAME"
echo "Transform job name: $TRANSFORM_JOB_NAME"
echo "Transform job input path: $TRANSFORM_S3_INPUT"
echo "Transform job output path: $TRANSFORM_S3_OUTPUT"

# Wait for the transform job to finish.
aws sagemaker wait transform-job-completed-or-stopped \
  --transform-job-name $TRANSFORM_JOB_NAME
  
# Examine the output.
aws s3 ls $TRANSFORM_S3_OUTPUT/000 --human-readable

# Copy an output example locally.
aws s3 cp $TRANSFORM_S3_OUTPUT/00000b4dcff7f799.jpg.out .

upload: ./model.tar.gz to s3://sagemaker-us-west-2-038453126632/sagemaker/DEMO-tf-batch-inference-jpeg-images-python-sdk/model/model.tar.gz
{
    "ModelArn": "arn:aws:sagemaker:us-west-2:038453126632:model/image-classification-tfs-2019-07-22-05-30-48"
}
{
    "TransformJobArn": "arn:aws:sagemaker:us-west-2:038453126632:transform-job/tfs-image-classification-job-2019-07-22-05-30-53"
}
Model name: image-classification-tfs-2019-07-22-05-30-48
Transform job name: tfs-image-classification-job-2019-07-22-05-30-53
Transform job input path: s3://sagemaker-sample-data-us-west-2/batch-transform/open-images/jpg
Transform job output path: s3://sagemaker-us-west-2-038453126632/sagemaker/DEMO-tf-batch-inference-jpeg-images-python-sdk/output
2019-07-22 05:35:13   12.7 KiB 00000b4dcff7f799.jpg.out
2019-07-22 05:35:13   12.6 KiB 00001a21632de752.jpg.out
2019-07-22 05:35:13   12.7 KiB 0000d67245642c5f.jpg.out
2019-07-22 05:35:13   12.6 KiB 0001244aa8ed3099.jpg.out
2019-07-22 05:35:13   12.7 KiB 000172d1

We see that after our transform job finishes, we find one S3 object in the output path for each object in the input path. This object contains the inferences from our model for that object, and has the same name as the corresponding input object, but with `.out` appended to it.

Inspecting one of the output objects, we find the prediction from our TensorFlow Serving model. This is from the example image displayed above:

In [7]:
!cat 00000b4dcff7f799.jpg.out

{
    "predictions": [
        {
            "probabilities": [7.48671e-07, 1.54555e-06, 3.04351e-06, 1.9618e-05, 6.92252e-06, 3.16003e-06, 1.96621e-05, 2.82172e-06, 3.88347e-05, 1.60989e-05, 1.53456e-05, 6.01257e-07, 2.78778e-06, 1.11208e-05, 7.98603e-07, 6.55138e-06, 3.26974e-06, 5.61108e-07, 5.62263e-06, 5.54361e-06, 2.101e-06, 9.41295e-07, 2.31893e-06, 3.48475e-06, 1.09363e-05, 4.11321e-06, 1.24613e-06, 9.51378e-07, 1.52576e-06, 1.21844e-06, 2.03722e-06, 1.32383e-06, 9.15461e-07, 1.98696e-06, 2.21267e-05, 5.08505e-06, 1.00016e-06, 2.03871e-06, 2.2516e-06, 5.01502e-07, 8.18207e-06, 9.78016e-07, 1.04662e-06, 1.79995e-06, 3.17814e-07, 1.0223e-06, 6.08684e-06, 6.15803e-07, 2.22039e-07, 1.18294e-05, 7.5845e-07, 4.56729e-06, 2.53492e-05, 2.34767e-06, 4.40761e-06, 5.03574e-06, 2.55696e-06, 2.91378e-06, 1.21964e-06, 2.48978e-06, 2.16968e-06, 1.89517e-05, 7.04592e-06, 4.72159e-06, 2.83891e-06, 1.44539e-06, 8.43768e-06, 7.74086e-07, 6.7491e-06, 1.4515e-06, 3.51813e-06, 5.45968e-07, 1.93929e-

## Conclusion

SageMaker batch transform can transform large datasets quickly and scalably. We used the SageMaker TensorFlow Serving Container to demonstrate how to quickly get inferences on a hundred thousand images using GPU-accelerated instances.

The Amazon SageMaker TFS container supports CSV and JSON data out of the box. The pre- and post-processing feature of the container lets you run transform jobs on data of any format. The same container can be used for real-time inference as well using an Amazon SageMaker hosted model endpoint.